In [1]:
import pandas as pd
from dfply import *
import matplotlib.pyplot as plt
import missingno as msno
from datetime import timedelta
import sys
sys.path.append('/Users/sdc/Documents/bida_lab/db/P1_extubationfailure/서대철')
import src.utils.utils as utils   # 기타 유틸리티 모듈
import src.data_extraction.access_database as db   # DB 연결 관련 모듈

In [2]:
# 변수를 붙일 subjectlist 데이터 가져오기

data_dir = '../outputs/subjectlist1.csv'
subjectlist = pd.read_csv(data_dir, index_col=0)

# datetime 변환
time_cols = ['admittime', 'intubationtime', 'extubationtime', 'dischtime', 'deathtime', 'reintubation_eventtime']
subjectlist = utils.to_datetime(subjectlist, time_cols)
print(subjectlist.shape)
subjectlist.head()

Converted admittime to datetime.
Converted intubationtime to datetime.
Converted extubationtime to datetime.
Converted dischtime to datetime.
Converted deathtime to datetime.
Converted reintubation_eventtime to datetime.
(9003, 27)


,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,...,reintubation_eventtime,reintubationtime,seq_num,mvtime,final_event,ext_to_death,ext_to_disch,disch_to_death,class_code,class
0,10001884,26184834,37510196.0,2131-01-07 20:39:00,2131-01-11 04:30:00,224385.0,65.0,37510196.0,2131-01-12 17:40:00,227194.0,...,NaT,NaN,1,False,True,10775.0,10775.0,0.0,11,nonfailure
1,10003400,23559586,38383343.0,2137-08-04 00:07:00,2137-08-17 21:21:00,224385.0,90.5,38383343.0,2137-08-21 15:40:00,227194.0,...,NaT,NaN,1,False,True,17365.0,17365.0,0.0,11,nonfailure
2,10004401,27939719,31202136.0,2144-04-11 03:31:00,2144-04-11 05:03:00,224385.0,120.0,NaN,2144-04-13 17:31:00,NaN,...,NaT,NaN,1,False,True,NaN,0.0,NaN,121,nonfailure
3,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-27 19:00:00,224385.0,76.0,32773003.0,2144-01-30 12:30:00,227194.0,...,2144-01-30 13:00:00,30.0,1,False,False,NaN,10035.0,NaN,211,failure
4,10004401,29988601,32773003.0,2144-01-23 07:58:00,2144-01-30 13:00:00,224385.0,76.0,NaN,2144-02-06 11:45:00,NaN,...,NaT,NaN,2,False,True,NaN,0.0,NaN,2221,nonfailure


In [3]:
# DB 연결 테이블 후 가져오기

database_config = {'database': 'mimic4',
 'user': 'postgres',
 'password': '1234',
 'host': 'localhost',
 'port': 5432}

# tables_query = {'pg_tables': 'SELECT * FROM PG_TABLES;',
#  'patients': 'SELECT * FROM mimiciv_hosp.patients;',
#  'd_items': 'SELECT * FROM mimiciv_icu.d_items;'}

tables_query = {'ventilation': "select * from mimiciv_derived.ventilation;"}

db.print_config_info(database_config, tables_query)
conn = db.connect_to_database(database_config)
dataframes = db.retrieve_data(conn, tables_query)
db.disconnect_database(conn)

# pg_tables = dataframes['pg_tables']
# patients = dataframes['patients']
# d_items = dataframes['d_items']

ventilation = dataframes['ventilation']

INFO:root:CONNECTING TO DATABASE...


--------- Database Configuration ---------
Database Name: mimic4
User: postgres
Password: ****
Host: localhost
Port: 5432

--------- SQL Queries for Required Tables ---------
ventilation: select * from mimiciv_derived.ventilation;



INFO:root:Retrieved ventilation: (111237, 4)
INFO:root:DATABASE CONNCETION CLOSED.


In [8]:
subjectlist.columns

Index(['subject_id', 'hadm_id', 'int_stayid', 'admittime', 'intubationtime',
       'int_itemid', 'int_weight', 'ext_stayid', 'extubationtime',
       'ext_itemid', 'ext_weight', 'extubationcause', 'dischtime', 'deathtime',
       'marker', 'reint_marker', 'intext_duration', 'reintubation_eventtime',
       'reintubationtime', 'seq_num', 'mvtime', 'final_event', 'ext_to_death',
       'ext_to_disch', 'disch_to_death', 'class_code', 'class'],
      dtype='object')

In [4]:
ventilation.columns

Index(['stay_id', 'starttime', 'endtime', 'ventilation_status'], dtype='object')

In [5]:
ventilation.head()

,stay_id,starttime,endtime,ventilation_status
0,30000153,2174-09-29 12:01:00,2174-09-29 20:00:00,InvasiveVent
1,30000153,2174-09-29 20:00:00,2174-10-01 00:57:00,SupplementalOxygen
2,30000213,2162-06-21 05:45:00,2162-06-21 17:35:00,InvasiveVent
3,30000484,2136-01-14 18:46:00,2136-01-17 04:00:00,SupplementalOxygen
4,30000646,2194-04-29 01:40:00,2194-05-03 16:00:00,SupplementalOxygen


In [30]:
test_subjectlist = subjectlist[subjectlist.reint_marker == True][['hadm_id', 'int_stayid', 'intubationtime', 'ext_stayid', 'extubationtime', 'intext_duration', 'seq_num', 'final_event', 'mvtime']]
test_subjectlist[test_subjectlist.hadm_id == 21588174]

,hadm_id,int_stayid,intubationtime,ext_stayid,extubationtime,intext_duration,seq_num,final_event,mvtime
8990,21588174,30916791.0,2166-06-14 08:58:00,30916791.0,2166-06-14 10:47:00,109.0,1,False,True
8991,21588174,31169972.0,2166-06-19 09:20:00,NaN,2166-06-19 21:15:00,715.0,2,True,True


In [20]:
ventilation[ventilation.stay_id == 30916791]

,stay_id,starttime,endtime,ventilation_status
10144,30916791,2166-06-13 16:36:00,2166-06-14 08:00:00,SupplementalOxygen
10145,30916791,2166-06-14 08:00:00,2166-06-14 12:00:00,InvasiveVent
10146,30916791,2166-06-14 12:00:00,2166-06-14 22:00:00,SupplementalOxygen
10147,30916791,2166-06-14 23:07:00,2166-06-16 12:00:00,SupplementalOxygen


In [17]:
test_subjectlist[test_subjectlist.mvtime == True]

,int_stayid,intubationtime,ext_stayid,extubationtime,intext_duration,mvtime
14,35889503.0,2150-03-27 07:44:00,35889503.0,2150-03-27 16:33:00,529.0,True
48,35381116.0,2115-10-11 14:05:00,35381116.0,2115-10-11 15:40:00,95.0,True
52,36886741.0,2192-12-15 22:30:00,36886741.0,2192-12-16 11:45:00,795.0,True
56,38669202.0,2169-02-10 16:22:00,38669202.0,2169-02-10 21:16:00,294.0,True
80,36809666.0,2184-08-20 08:42:00,36809666.0,2184-08-20 11:30:00,168.0,True
...,...,...,...,...,...,...
8960,32028235.0,2145-06-03 04:00:00,NaN,2145-06-03 05:16:00,76.0,True
8975,33646490.0,2122-01-13 01:00:00,33646490.0,2122-01-13 03:11:00,131.0,True
8990,30916791.0,2166-06-14 08:58:00,30916791.0,2166-06-14 10:47:00,109.0,True
8991,31169972.0,2166-06-19 09:20:00,NaN,2166-06-19 21:15:00,715.0,True


In [36]:
# raw data를 보자
# 변수를 붙일 subjectlist 데이터 가져오기

data_dir = '/Users/sdc/Documents/bida_lab/db/P1_extubationfailure/이수정/mimic4_ext_codes/data/intubation_extubation.csv'
raw_vent = pd.read_csv(data_dir)

# datetime 변환
time_cols = ['admittime', 'intubationtime', 'extubationtime', 'dischtime', 'deathtime']
raw_vent = utils.to_datetime(raw_vent, time_cols)
print(raw_vent.shape)
raw_vent.head()

Converted admittime to datetime.
Converted intubationtime to datetime.
Converted extubationtime to datetime.
Converted dischtime to datetime.
Converted deathtime to datetime.
(10992, 14)


,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime
0,10001884,26184834,37510196,2131-01-07 20:39:00,2131-01-11 04:30:00,224385,65.0,37510196.0,2131-01-12 17:40:00,227194.0,65.0,Planned Extubation,2131-01-20 05:15:00,2131-01-20 05:15:00
1,10003400,23559586,38383343,2137-08-04 00:07:00,2137-08-17 21:21:00,224385,90.5,38383343.0,2137-08-21 15:40:00,227194.0,99.6,Planned Extubation,2137-09-02 17:05:00,2137-09-02 17:05:00
2,10004401,27939719,31202136,2144-04-11 03:31:00,2144-04-11 05:03:00,224385,120.0,NaN,NaT,NaN,NaN,NaN,2144-04-13 17:31:00,NaT
3,10004401,29988601,32773003,2144-01-23 07:58:00,2144-01-27 19:00:00,224385,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaT
4,10004401,29988601,32773003,2144-01-23 07:58:00,2144-01-30 13:00:00,224385,76.0,32773003.0,2144-01-30 12:30:00,227194.0,76.0,Planned Extubation,2144-02-06 11:45:00,NaT


In [37]:
raw_vent[raw_vent.int_stayid == 30916791]

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,ext_weight,extubationcause,dischtime,deathtime
10968,19983257,21588174,30916791,2166-06-13 14:21:00,2166-06-14 08:58:00,224385,118.6,30916791.0,2166-06-14 10:47:00,227194.0,118.6,Planned Extubation,2166-06-19 21:15:00,2166-06-19 21:15:00



---

In [40]:
event_type = ['Tracheostomy', 'InvasiveVent', 'HFNC']

vent_filtered = ventilation[ventilation.ventilation_status.isin(event_type)]
vent_filtered

,stay_id,starttime,endtime,ventilation_status
0,30000153,2174-09-29 12:01:00,2174-09-29 20:00:00,InvasiveVent
2,30000213,2162-06-21 05:45:00,2162-06-21 17:35:00,InvasiveVent
5,30001148,2156-08-30 14:50:00,2156-08-30 18:00:00,InvasiveVent
12,30001656,2177-03-15 17:32:00,2177-03-16 12:00:00,InvasiveVent
14,30001947,2162-12-26 19:00:00,2162-12-27 09:19:00,InvasiveVent
...,...,...,...,...
111224,39997753,2110-09-11 23:00:00,2110-09-12 14:00:00,InvasiveVent
111226,39998012,2133-02-03 15:00:00,2133-02-08 16:00:00,InvasiveVent
111227,39998622,2135-02-11 18:15:00,2135-02-14 08:00:00,InvasiveVent
111228,39998622,2135-02-17 09:00:00,2135-02-18 08:00:00,InvasiveVent


In [46]:
# 환자 데이터에 포함된 stayid만 추출

unique_intstay = subjectlist.int_stayid.unique()
unique_extstay = subjectlist.ext_stayid.unique()

# Convert each list to a set
set_intstay = set(unique_intstay)
set_extstay = set(unique_extstay)

# Use set union to combine and deduplicate
combined_unique_stay = set_intstay.union(set_extstay)

# Convert back to a list if you need a list structure
combined_unique_stay_list = list(combined_unique_stay)

# combined_unique_stay_list now contains unique values from both original lists
len(combined_unique_stay_list)

vent_filtered = vent_filtered[vent_filtered.stay_id.isin(combined_unique_stay_list)]
vent_filtered.shape

(12401, 4)

In [54]:
vent_filtered.head()

,stay_id,starttime,endtime,ventilation_status
5,30001148,2156-08-30 14:50:00,2156-08-30 18:00:00,InvasiveVent
12,30001656,2177-03-15 17:32:00,2177-03-16 12:00:00,InvasiveVent
14,30001947,2162-12-26 19:00:00,2162-12-27 09:19:00,InvasiveVent
32,30004144,2126-04-05 16:00:00,2126-04-06 17:00:00,InvasiveVent
33,30004144,2126-04-06 18:00:00,2126-04-07 08:00:00,InvasiveVent


In [48]:
subjectlist[subjectlist.int_stayid == 39991569]

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,...,reintubation_eventtime,reintubationtime,seq_num,mvtime,final_event,ext_to_death,ext_to_disch,disch_to_death,class_code,class
3102,13439409,22868166,39991569.0,2179-04-10 14:39:00,2179-04-10 17:25:00,224385.0,94.2,39991569.0,2179-04-11 16:18:00,227194.0,...,NaT,NaN,1,True,True,NaN,11502.0,NaN,11,nonfailure


In [49]:
vent_filtered[vent_filtered.stay_id == 39991569]

,stay_id,starttime,endtime,ventilation_status
111180,39991569,2179-04-10 17:22:00,2179-04-11 20:00:00,InvasiveVent


In [52]:
subjectlist[subjectlist.ext_stayid == 39995735]

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,...,reintubation_eventtime,reintubationtime,seq_num,mvtime,final_event,ext_to_death,ext_to_disch,disch_to_death,class_code,class
1384,11539827,21192405,NaN,2124-08-05 14:24:00,NaT,NaN,NaN,39995735.0,2124-08-21 06:17:00,227194.0,...,2124-08-23 10:30:00,3133.0,1,False,False,NaN,22131.0,NaN,212,nonfailure


In [53]:
vent_filtered[vent_filtered.stay_id == 39995735]

,stay_id,starttime,endtime,ventilation_status
111204,39995735,2124-08-15 18:00:00,2124-08-21 09:58:00,InvasiveVent


In [55]:
subjectlist[subjectlist.int_stayid == 30004144]

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,...,reintubation_eventtime,reintubationtime,seq_num,mvtime,final_event,ext_to_death,ext_to_disch,disch_to_death,class_code,class
335,10369174,24697158,30004144.0,2126-04-04 08:00:00,2126-04-05 16:00:00,224385.0,80.0,30004144.0,2126-04-06 13:00:00,227194.0,...,NaT,NaN,1,True,True,NaN,7142.0,NaN,11,nonfailure


In [56]:
vent_filtered[vent_filtered.stay_id == 30004144]

,stay_id,starttime,endtime,ventilation_status
32,30004144,2126-04-05 16:00:00,2126-04-06 17:00:00,InvasiveVent
33,30004144,2126-04-06 18:00:00,2126-04-07 08:00:00,InvasiveVent


In [59]:
subjectlist[(subjectlist.intubationtime.isna()) | (subjectlist.extubationtime.isna())]

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,...,reintubation_eventtime,reintubationtime,seq_num,mvtime,final_event,ext_to_death,ext_to_disch,disch_to_death,class_code,class
12,10020740,23831430,NaN,2150-03-11 15:34:00,NaT,NaN,NaN,35026312.0,2150-03-12 18:15:00,227194.0,...,NaT,NaN,1,False,False,NaN,63095.0,NaN,9999,NaN
13,10020740,23831430,NaN,2150-03-11 15:34:00,NaT,NaN,NaN,35044342.0,2150-03-19 11:05:00,227194.0,...,2150-03-27 07:44:00,11319.0,2,False,False,NaN,53445.0,NaN,212,nonfailure
18,10024982,25154057,NaN,2203-09-19 12:29:00,NaT,NaN,NaN,37919901.0,2203-09-28 17:00:00,227194.0,...,2203-10-02 11:00:00,5400.0,1,False,False,18850.0,18850.0,0.0,212,nonfailure
21,10027602,28166872,NaN,2201-10-30 12:05:00,NaT,NaN,NaN,32391858.0,2201-11-07 13:53:00,227194.0,...,2201-11-07 15:00:00,67.0,1,False,False,NaN,18772.0,NaN,211,failure
47,10079700,24239751,38841663.0,2115-09-14 22:01:00,2115-09-27 17:23:00,224385.0,64.0,NaN,NaT,NaN,...,2115-10-11 14:05:00,NaN,1,False,False,NaN,NaN,NaN,9999,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8977,19969918,29544887,31213286.0,2186-01-15 23:00:00,2186-01-16 20:29:00,224385.0,68.0,NaN,NaT,NaN,...,2186-01-16 21:10:00,NaN,1,False,False,NaN,NaN,NaN,998,failure
8986,19975796,25848942,NaN,2148-12-02 22:08:00,NaT,NaN,NaN,31651964.0,2148-12-03 10:33:00,227194.0,...,2148-12-04 20:15:00,2022.0,1,False,False,NaN,16167.0,NaN,211,failure
8997,19989918,20256395,NaN,2175-09-18 17:45:00,NaT,NaN,NaN,35582926.0,2175-09-26 16:00:00,227194.0,...,NaT,NaN,2,False,True,NaN,2985.0,NaN,221,nonfailure
9000,19995595,21784060,NaN,2126-10-17 21:43:00,NaT,NaN,NaN,34670930.0,2126-10-30 14:26:00,227194.0,...,2126-10-31 00:45:00,619.0,1,False,False,NaN,18909.0,NaN,211,failure


In [60]:
subjectlist[subjectlist.ext_stayid == 35026312.0]

,subject_id,hadm_id,int_stayid,admittime,intubationtime,int_itemid,int_weight,ext_stayid,extubationtime,ext_itemid,...,reintubation_eventtime,reintubationtime,seq_num,mvtime,final_event,ext_to_death,ext_to_disch,disch_to_death,class_code,class
12,10020740,23831430,NaN,2150-03-11 15:34:00,NaT,NaN,NaN,35026312.0,2150-03-12 18:15:00,227194.0,...,NaT,NaN,1,False,False,NaN,63095.0,NaN,9999,NaN


In [61]:
vent_filtered[vent_filtered.stay_id == 35026312]

,stay_id,starttime,endtime,ventilation_status
56233,35026312,2150-03-11 16:00:00,2150-03-12 19:00:00,InvasiveVent
